<a href="https://colab.research.google.com/github/sarthak395/Dass_Project/blob/main/Memory/Memorymodelextract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np


In [2]:
# LOADING DATA
def load_data(input_file):
    data = pd.read_csv(input_file)
    return data


In [3]:
def inputcols(input_dataset):
    names = input_dataset['Name']
    details = input_dataset['Details']
    return names, details


In [11]:
data = load_data('memory.csv')
names, details = inputcols(data)


In [5]:
def extract_adata_name(names,expression):
    adatalist_name = []
    for name in names:
        re_search = re.search(expression, name)
        if re_search:
            adatalist_name.append(name)
    return adatalist_name

In [6]:
def extract_adata_details(details,expression):
    adatalist_det = []
    for det in details:
        re_search = re.search(expression, det)
        if re_search:
            adatalist_det.append(det)
    return adatalist_det;

In [16]:
def filtering_adata_products(names, details, expression):
    adatalist_name = extract_adata_name(names, expression);
    adatalist_det = extract_adata_details(details, expression);
    adatalist = pd.concat([pd.Series(adatalist_name), pd.Series(adatalist_det)], axis=1)
    return adatalist;

In [19]:
def extract__adata_model_data(names,details,adatalist,model_expression):
    adatamodelnumbers = []

    for name, det in adatalist.itertuples(index=False):
        re_search = re.search(model_expression, name)
        if re_search:
            adatamodelnumbers.append({"company": "adata", "name": name, "model": re_search.group(0)})
        else:
            re_search2 = re.search(model_expression, det)
            if re_search2:
                adatamodelnumbers.append({"company": "adata", "name": name, "model": re_search2.group(0)})
            else:
                adatamodelnumbers.append({"company": "adata", "name": name, "model": np.nan})

    # NOW CHECK

    adatamodelnumbers = pd.DataFrame(adatamodelnumbers)
    return adatamodelnumbers;

In [17]:
class Adata:
    def __init__(self,expression,model_expression,names,details):
        self.names = names;
        self.details = details;
        self.expression = expression;
        self.model_expression = model_expression;
        self.adatalist = pd.DataFrame()
        self.adata_data = []
    
    def setadatalist(self):
        self.adatalist = filtering_adata_products(self.names, self.details, self.expression)
    
    def setadata_data(self):
        self.adata_data = extract__adata_model_data(self.names,self.details,self.adatalist,self.model_expression)

In [ ]:
adata = Adata('(?i)adata ' , '(AX4U|AD4U|AX4S)[A-Za-z0-9-]*( )?' , names , details)
adata.setadatalist()
adata.setadata_data()

In [58]:
# EXTRACTED MODEL NUMBERS FOR ADATA
expression = '(?i)adata '
adatalist_name = []
adatalist_det = []
for name in names:
    inch_search = re.search(expression, name)
    if inch_search:
        adatalist_name.append(name)

for det in details:
    inch_search = re.search(expression, det)
    if inch_search:
        adatalist_det.append(det)

adatalist = pd.concat(
    [pd.Series(adatalist_name), pd.Series(adatalist_det)], axis=1)
adatamodelnumbers = []


# now we have to find model no.
model_expression = '(AX4U|AD4U|AX4S)[A-Za-z0-9-]*( )?'
for name, det in adatalist.itertuples(index=False):
    inch_search = re.search(model_expression, name)
    if inch_search:
      # sizesearch = re.search(size_expression,name)
      # freqsearch = re.search(freq_expression,name)
      # if freqsearch:
        adatamodelnumbers.append(
            {"company": "adata", "name": name, "model": inch_search.group(0)})
      # else:
      #   print(name)
    else:
        inch_search2 = re.search(model_expression, det)
        if inch_search2:
            # sizesearch = re.search(size_expression,name)
            # freqsearch = re.search(freq_expression,name)
            # if freqsearch:
            adatamodelnumbers.append(
                {"company": "adata", "name": name, "model": inch_search2.group(0)})
        # else:
        #   print(det)
        else:
            adatamodelnumbers.append(
                {"company": "adata", "name": name, "model": np.nan})

# NOW CHECK

adatamodelnumbers = pd.DataFrame(adatamodelnumbers)


In [ ]:
print(adatamodelnumbers)


In [60]:
# EXTRACTED MODEL NUMBERS FOR CRUCIAL
expression = '(?i)crucial '
cruciallist_name = []
cruciallist_det = []
for name in names:
    inch_search = re.search(expression, name)
    if inch_search:
        cruciallist_name.append(name)

for det in details:
    inch_search = re.search(expression, det)
    if inch_search:
        cruciallist_det.append(det)

cruciallist = pd.concat(
    [pd.Series(cruciallist_name), pd.Series(cruciallist_det)], axis=1)
crucialmodelnumbers = []
# now we have to find model no.
model_expression = '(BL|CB|CT)[A-Za-z0-9-]*( )?'
for name, det in cruciallist.itertuples(index=False):
    inch_search = re.search(model_expression, name)
    if inch_search:
        crucialmodelnumbers.append(
            {"company": "crucial", "name": name, "model": inch_search.group(0)})
    else:
        inch_search2 = re.search(model_expression, det)
        if inch_search2:
            crucialmodelnumbers.append(
                {"company": "crucial", "name": name, "model": inch_search2.group(0)})
        else:
            crucialmodelnumbers.append(
                {"company": "crucial", "name": name, "model": np.nan})

# NOW CHECK

crucialmodelnumbers = pd.DataFrame(crucialmodelnumbers)


In [61]:
print(crucialmodelnumbers)


   company                                               name           model
0  crucial  Crucial Ballistix 16GB Kit (2 x 8GB) DDR4 3000...  BL2K8G30C15U4B
1  crucial  Crucial Ballistix Sport LT 16GB 3200 MHz DDR4 ...  BLS16G4D32AESC
2  crucial  Crucial Ballistix Sport LT 8GB 2666 MHz DDR4 R...  BLS8G4D26BFSEK
3  crucial  Crucial Ballistix Sport LT 8GB 3200 MHz DDR4 G...  BLS8G4D32AESBK
4  crucial  Crucial 16GB 2666MHz DDR4 SODIMM 260 Pin Lapto...      CB16GS2666
5  crucial  Crucial 4GB DDR4 2400MHZ SODIMM Laptop Memory ...       CB4GS2400
6  crucial      Crucial 4GB DDR4 2400 MHz UDIMM RAM CB4GU2400       CB4GU2400
7  crucial  Crucial 8GB DDR4 2400MHZ SODIMM Laptop Memory ...       CB8GS2400
8  crucial  Crucial Basics 8GB DDR4 2400MHZ UDIMM Desktop ...       CB8GU2400
9  crucial   Crucial 4GB DDR4 2400 MHz UDIMM RAM CT4G4DFS824A    CT4G4DFS824A


In [62]:
# EXTRACTED MODEL NUMBERS FOR CORCAIR
expression = '(?i)corsair '
corcairlist_name = []
corcairlist_det = []
for name in names:
    inch_search = re.search(expression, name)
    if inch_search:
        corcairlist_name.append(name)

for det in details:
    inch_search = re.search(expression, det)
    if inch_search:
        corcairlist_det.append(det)

corcairlist = pd.concat(
    [pd.Series(corcairlist_name), pd.Series(corcairlist_det)], axis=1)
corcairmodelnumbers = []
# now we have to find modorcair
model_expression = '(CMD|CMK|CMZ|CMW|CMT|CMV)[A-Za-z0-9-]*( )?'
for name, det in corcairlist.itertuples(index=False):
    inch_search = re.search(model_expression, name)
    if inch_search:
        corcairmodelnumbers.append(
            {"company": "corcair", "name": name, "model": inch_search.group(0)})
    else:
        inch_search2 = re.search(model_expression, det)
        if inch_search2:
            corcairmodelnumbers.append(
                {"company": "corcair", "name": name, "model": inch_search2.group(0)})
        else:
            corcairmodelnumbers.append(
                {"company": "corcair", "name": name, "model": np.nan})

# NOW CHECK

corcairmodelnumbers = pd.DataFrame(corcairmodelnumbers)


In [ ]:
print(corcairmodelnumbers)


In [64]:
# EXTRACTED MODEL NUMBERS FOR g.skill
expression = '(?i)(g.skill) '
gskilllist_name = []
gskilllist_det = []
for name in names:
    inch_search = re.search(expression, name)
    if inch_search:
        gskilllist_name.append(name)

for det in details:
    inch_search = re.search(expression, det)
    if inch_search:
        gskilllist_det.append(det)

gskilllist = pd.concat(
    [pd.Series(gskilllist_name), pd.Series(gskilllist_det)], axis=1)
gskillmodelnumbers = []
# now we have to find modorcair
model_expression = '(F4-)[A-Za-z0-9-]*( )?'
for name, det in gskilllist.itertuples(index=False):
    inch_search = re.search(model_expression, name)
    if inch_search:
        gskillmodelnumbers.append(
            {"company": "gskill", "name": name, "model": inch_search.group(0)})
    else:
        inch_search2 = re.search(model_expression, det)
        if inch_search2:
            gskillmodelnumbers.append(
                {"company": "gskill", "name": name, "model": inch_search2.group(0)})
        else:
            gskillmodelnumbers.append(
                {"company": "gskill", "name": name, "model": np.nan})

# NOW CHECK

gskillmodelnumbers = pd.DataFrame(gskillmodelnumbers)


In [65]:
print(gskillmodelnumbers)


   company                                               name  \
0   gskill  G.SKILL Aegis Series 32GB (4x8GB) 2133MHz DDR4...   
1   gskill  G.Skill F4-2400C15S-16GVR Desktop Ram Ripjaws ...   
2   gskill  G.Skill F4-2400C16S-16GRS Laptop Ram Ripjaws S...   
3   gskill  G.Skill F4-2400C16S-4GRS Laptop Ram Ripjaws Se...   
4   gskill  G.Skill F4-2400C16S-8GRS Laptop Ram Ripjaws Se...   
..     ...                                                ...   
73  gskill  G.Skill F4-4000C18D-16GTZRB Desktop Ram Triden...   
74  gskill  G.Skill RIPJAWS V 16GB (8GBX2) DDR4 4000MHz Me...   
75  gskill  G.SKILL F4-4000C19D-32GTZR Desktop Ram Trident...   
76  gskill  G.Skill TRIDENT Z 16GB (8GBX2) DDR4 4133MHz RG...   
77  gskill  G.Skill F4-4266C19D-16GTZR Desktop Ram Trident...   

                   model  
0      F4-2133C15Q-32GIS  
1     F4-2400C15S-16GVR   
2     F4-2400C16S-16GRS   
3      F4-2400C16S-4GRS   
4      F4-2400C16S-8GRS   
..                   ...  
73  F4-4000C18D-16GTZRB   
74 

In [66]:
# EXTRACTED MODEL NUMBERS FOR antec
expression = '(?i)(antec) '
anteclist_name = []
anteclist_det = []
for name in names:
    inch_search = re.search(expression, name)
    if inch_search:
        anteclist_name.append(name)

for det in details:
    inch_search = re.search(expression, det)
    if inch_search:
        anteclist_det.append(det)

anteclist = pd.concat(
    [pd.Series(anteclist_name), pd.Series(anteclist_det)], axis=1)
antecmodelnumbers = []
# now we have to find modorcair
model_expression = '(AM)[A-Za-z0-9-]*( )?'
for name, det in anteclist.itertuples(index=False):
    inch_search = re.search(model_expression, name)
    if inch_search:
        antecmodelnumbers.append(
            {"company": "antec", "name": name, "model": inch_search.group(0)})
    else:
        inch_search2 = re.search(model_expression, det)
        if inch_search2:
            antecmodelnumbers.append(
                {"company": "antec", "name": name, "model": inch_search2.group(0)})
        else:
            antecmodelnumbers.append(
                {"company": "antec", "name": name, "model": np.nan})

# NOW CHECK
antecmodelnumbers = pd.DataFrame(antecmodelnumbers)


In [67]:
# EXTRACTED MODEL NUMBERS FOR gigabyte
expression = '(?i)(gigabyte) '
gigabytelist_name = []
gigabytelist_det = []
for name in names:
    inch_search = re.search(expression, name)
    if inch_search:
        gigabytelist_name.append(name)

for det in details:
    inch_search = re.search(expression, det)
    if inch_search:
        gigabytelist_det.append(det)

gigabytelist = pd.concat(
    [pd.Series(gigabytelist_name), pd.Series(gigabytelist_det)], axis=1)
gigabytemodelnumbers = []
# now we have to find modorcair
model_expression = '(GP-)[A-Za-z0-9-]*( )?'
for name, det in gigabytelist.itertuples(index=False):
    inch_search = re.search(model_expression, name)
    if inch_search:
        gigabytemodelnumbers.append(
            {"company": "gigabyte", "name": name, "model": inch_search.group(0)})
    else:
        inch_search2 = re.search(model_expression, det)
        if inch_search2:
            gigabytemodelnumbers.append(
                {"company": "gigabyte", "name": name, "model": inch_search2.group(0)})
        else:
            print(name)

# NOW CHECK
gigabytemodelnumbers = pd.DataFrame(gigabytemodelnumbers)


In [68]:
print(gigabytemodelnumbers)


    company                                               name  \
0  gigabyte  GIGABYTE GP-AR32C16S8K2HU416R Desktop Ram 16GB...   
1  gigabyte  Gigabyte GP-AR36C18S8K2HU416R Desktop Ram Aoru...   
2  gigabyte  Gigabyte AORUS RGB 16GB (8GBx2) DDR4 4400MHz (...   

                   model  
0  GP-AR32C16S8K2HU416R   
1  GP-AR36C18S8K2HU416R   
2            GP-ARS16G44  


In [83]:
# Remaining
remainingmodelnumbers = pd.DataFrame(
    [{"company": "Thermaltake", "name": "Thermaltake Desktop WaterRam RGB Liquid Cooling Memory 16GB (8GBx2) DDR4 3200MHz", "model": np.nan}])


In [98]:
result = pd.concat([gigabytemodelnumbers, antecmodelnumbers, gskillmodelnumbers,
                   corcairmodelnumbers, crucialmodelnumbers, adatamodelnumbers], ignore_index=True)
size = []
freq = []
# Iterate over result and add size and frequency
size_expression = '[0-9]{1,2}( )?GB( )?'
freq_expression = '[0-9]{4}( )?(?i)MHz( )?'
for ind in result.index:
    sizesearch = re.search(size_expression, result['name'][ind])
    freqsearch = re.search(freq_expression, result['name'][ind])
    if sizesearch:
        size.append(sizesearch.group(0))
    else:
        size.append(np.nan)
    if freqsearch:
        freq.append(freqsearch.group(0))
    else:
        freq.append(np.nan)
result['Size'] = size
result['Frequency'] = freq
result.to_csv('Memory_models.csv')


In [ ]:
print(result)
